The inference code in the "sagemaker-staging" folder outputs a json file. This code shows how to convert that to a geojson file.

In [2]:
from geojson import Polygon, Feature, FeatureCollection, dump

In [3]:
import json

## You can find the json files in the "raw_predictions" zip folder

with open('SAB_2021_results.json', 'r') as openfile:
    d = json.load(openfile)

In [6]:
# Get the value of the first key
d[list(d.keys())[0]]

[{'window_id': 1,
  'polygon_coords': [[16.206865366913934, -2.0974763568906707],
   [16.206865366913934, -2.0884932040494757],
   [16.197882214072738, -2.0884932040494757],
   [16.197882214072738, -2.0974763568906707],
   [16.206865366913934, -2.0974763568906707]],
  'labels': []},
 {'window_id': 2,
  'polygon_coords': [[16.215848519755127, -2.0974763568906707],
   [16.215848519755127, -2.0884932040494757],
   [16.206865366913934, -2.0884932040494757],
   [16.206865366913934, -2.0974763568906707],
   [16.215848519755127, -2.0974763568906707]],
  'labels': ['Shifting_cultivation']},
 {'window_id': 3,
  'polygon_coords': [[16.224831672596324, -2.0974763568906707],
   [16.224831672596324, -2.0884932040494757],
   [16.215848519755127, -2.0884932040494757],
   [16.215848519755127, -2.0974763568906707],
   [16.224831672596324, -2.0974763568906707]],
  'labels': []},
 {'window_id': 4,
  'polygon_coords': [[16.23381482543752, -2.0974763568906707],
   [16.23381482543752, -2.0884932040494757],


In [7]:
test_coords = d[list(d.keys())[0]][0]['polygon_coords']

In [8]:
test_coords

[[16.206865366913934, -2.0974763568906707],
 [16.206865366913934, -2.0884932040494757],
 [16.197882214072738, -2.0884932040494757],
 [16.197882214072738, -2.0974763568906707],
 [16.206865366913934, -2.0974763568906707]]

In [9]:
# Put coordinates in a geojson Polygon

Polygon(test_coords)

{"coordinates": [[16.206865, -2.097476], [16.206865, -2.088493], [16.197882, -2.088493], [16.197882, -2.097476], [16.206865, -2.097476]], "type": "Polygon"}

In [4]:
SAB_polys = []

for rast in list(d.keys()): # each key corresponds to one raster
    windows = d[rast] # the value is a list of each 1 km x 1 km window
    for window in windows:
        if 'SAB' in window['labels']:
            # only pull out the slash and burn predictions
            SAB_polys.append(window['polygon_coords'])

In [5]:
len(SAB_polys)

164152

In [6]:
features = []

for poly in SAB_polys:
    polygon = Polygon([poly]) # need to put it in another list for the function to work
    features.append(Feature(geometry=polygon))
    
crs = { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } } # define CRS

feature_collection = FeatureCollection(features, crs=crs)

with open('SAB_2021_preds.geojson', 'w') as f:
   dump(feature_collection, f) # dump into a geojson

In [ ]:
def convert_json_to_geojson(json_loc, geojson_loc, label, crs_name):
    """
    Function for converting a json file with predictions to a geojson file with only one type of prediction.
    Arguments:
    json_loc: The location of the json file. This should be the output of the inference code found in
    the "sagemaker-staging" folder.
    geojson_loc: The location/filename to save the geojson file to.
    label: Which specific label to pull out for the geojson file.
    crs_name: The name to give the CRS in the geojson file.
    """
    
    with open(json_loc, 'r') as openfile:
        d = json.load(openfile)
        
    polys = []
    
    for rast in list(d.keys()): # each key corresponds to one raster
        windows = d[rast] # the value is a list of each 1 km x 1 km window
        for window in windows:
            if label in window['labels']:
                # only pull out the appropriate predictions
                polys.append(window['polygon_coords'])
                
    features = []

    for poly in polys:
        polygon = Polygon([poly]) # need to put it in another list for the function to work
        features.append(Feature(geometry=polygon))

    crs = {"type": "name", "properties": {"name": crs_name}} # define CRS

    feature_collection = FeatureCollection(features, crs=crs)

    with open(geojson_loc, 'w') as f:
       dump(feature_collection, f) # dump into a geojson

In [ ]:
json_base = './raw_predictions/'
json_filenames = [
    'ISL_2019_results.json', 'ISL_2021_results.json', 'SAB_2019_results.json', 'SAB_2021_results.json'
]

for filename in json_filenames:
    json_path = json_base + filename
    f_split = filename.split('_')
    geojson_path = './predictions/' + f_split[0] + '_' + f_split[1] + '_preds.geojson'
    label = f_split[0]
    crs_name = "urn:ogc:def:crs:OGC:1.3:CRS84"
    convert_json_to_geojson(json_path, geojson_path, label, crs_name)